In [1]:
import csv
train, dev, test = [], [], []

In [2]:
with open('pnli_train.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[2] will be the label (0 or 1). x[0] and x[1] will be the sentence pairs.
        train.append(x)
print (len(train))
print (train[:3])

5983
[['Sometimes do exercise.', 'A person typically desire healthy life.', '1'], ['Who eats junk foods.', 'A person typically desire healthy life.', '0'], ['A person is sick.', 'A person typically desire healthy life.', '1']]


In [3]:
with open('pnli_dev.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[2] will be the label (0 or 1). x[0] and x[1] will be the sentence pairs.
        dev.append(x)
print (len(dev))
print (dev[:3])

1055
[['A person is looking for accuracy.', 'A person typically desires accurate results.', '1'], ['A person does not care for accuracy.', 'A person typically desires accurate results.', '0'], ['The person double checks their data.', 'A person typically desires accurate results.', '1']]


In [4]:
with open('pnli_test_unlabeled.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[0] and x[1] will be the sentence pairs.
        test.append(x)
print (len(test))
print (test[:3])

4850
[['The people want to have a romantic and pleasant feel.', 'People typically does desire to smell violets.'], ['The contract is to buy products from you.', 'Getting contract typically cause to make money or spend money.'], ['Train station is closed.', 'Line can typically be used to move train along tracks.']]


In [ ]:
import pandas as pd
train_data=pd.DataFrame(train,columns=["precondition","statement","labels"])
c=0
labels=train_data["labels"]
labels=list(labels)
for i in range(len(labels)):
  if(labels[i]=="1"):
    labels[i]="entailment"
  else:
    labels[i]="contradiction"
train_data["label_val"]=labels
train_data.head()

In [6]:
train_data["labels"] = train_data["labels"].astype(str).astype(int)

In [ ]:
dev_data=pd.DataFrame(dev,columns=["precondition","statement","labels"])
dev_data.head()
labels=dev_data["labels"]
labels=list(labels)
for i in range(len(labels)):
  if(labels[i]=="1"):
    labels[i]="entailment"
  else:
    labels[i]="contradiction"
dev_data["label_val"]=labels
dev_data.head()

In [8]:
dev_data["labels"] = dev_data["labels"].astype(str).astype(int)

In [9]:
test_data=pd.DataFrame(test,columns=["precondition","statement"])
test_data.head()

,precondition,statement
0,The people want to have a romantic and pleasan...,People typically does desire to smell violets.
1,The contract is to buy products from you.,Getting contract typically cause to make money...
2,Train station is closed.,Line can typically be used to move train along...
3,There is no water for driving the boats.,People typically desires drive boats for fun.
4,The poet is busy.,Poet can typically be used for creating poetry.


In [10]:
!pip install --no-cache-dir transformers sentencepiece

     |████████████████████████████████| 4.0 MB 29.0 MB/s 
     |████████████████████████████████| 1.2 MB 71.6 MB/s 
     |████████████████████████████████| 895 kB 63.9 MB/s 
     |████████████████████████████████| 77 kB 69.7 MB/s 
     |████████████████████████████████| 596 kB 78.5 MB/s 
     |████████████████████████████████| 6.6 MB 48.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
from transformers import BertTokenizer, TFBertModel, TFAutoModel,AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base",use_fast=False)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [12]:
def encode_premise_sentence(s):
    tokens=[]
    tokens.append('[CLS]')
    tokens+=list(tokenizer.tokenize(s))
    return tokenizer.convert_tokens_to_ids(tokens)
encode_premise_sentence("paras")

[3, 5489, 281]

In [13]:
def encode_hypo_sentence(s):
    tokens=[]
    tokens.append('[sep]')
    tokens+=list(tokenizer.tokenize(s))
    tokens.append('[sep]')
    return tokenizer.convert_tokens_to_ids(tokens)
encode_premise_sentence("tusharmittal")

[3, 90, 3810, 4526, 20743]

In [ ]:
tokenized=[]
for i in range(len(train_data)):
    pre=encode_premise_sentence(train_data['precondition'][i])
    hyp=encode_hypo_sentence(train_data['statement'][i])
    tokenized.append(pre+hyp)
train_data['tokenized']=tokenized
train_data.head()

In [ ]:
tokenized=[]
for i in range(len(dev_data)):
    pre=encode_premise_sentence(dev_data['precondition'][i])
    hyp=encode_hypo_sentence(dev_data['statement'][i])
    tokenized.append(pre+hyp)
dev_data['tokenized']=tokenized
dev_data.head()

In [16]:
mask=[]
for i in range(len(train_data)):
    padded_seq=tokenizer(train_data['precondition'][i],train_data['statement'][i], padding=True,add_special_tokens = True)
    mask.append(padded_seq)

In [17]:
df_train_panda = train_data
df_train_panda['masked'] = mask
df_train_panda.head(5)

,precondition,statement,labels,label_val,tokenized,masked
0,Sometimes do exercise.,A person typically desire healthy life.,1,entailment,"[3, 13624, 109, 3325, 4, 3, 250, 621, 3700, 47...","[input_ids, attention_mask]"
1,Who eats junk foods.,A person typically desire healthy life.,0,contradiction,"[3, 12375, 24923, 15163, 6592, 4, 3, 250, 621,...","[input_ids, attention_mask]"
2,A person is sick.,A person typically desire healthy life.,1,entailment,"[3, 250, 621, 16, 4736, 4, 3, 250, 621, 3700, ...","[input_ids, attention_mask]"
3,A person is dead.,A person typically desire healthy life.,0,contradiction,"[3, 250, 621, 16, 1462, 4, 3, 250, 621, 3700, ...","[input_ids, attention_mask]"
4,A person eats properly and do exercise regularly.,A person typically desire healthy life.,1,entailment,"[3, 250, 621, 24923, 5083, 8, 109, 3325, 4595,...","[input_ids, attention_mask]"


In [ ]:
mask=[]
for i in range(len(dev_data)):
    padded_seq=tokenizer(dev_data['precondition'][i],dev_data['statement'][i], padding=True,add_special_tokens = True)
    mask.append(padded_seq)
df_dev_panda = dev_data
df_dev_panda['masked'] = mask
df_dev_panda.head(5)

In [19]:
import tensorflow as tf

In [35]:
max_len = 31

def build_model():
    encoder = TFAutoModel.from_pretrained('roberta-base')
  
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")  
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
 
    embedding = encoder([input_word_ids, input_mask])[0] 
    output = tf.keras.layers.Dense(2, activation='softmax')(embedding[:,0,:])
    model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs=output) 
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy']) 
    
    return model

In [36]:
def input_convert(data):
    inputs={
        'input_word_ids':[],
        'input_mask':[]
    }
    for each in data:
        inputs['input_word_ids'].append(each['input_ids'])
        inputs['input_mask'].append(each['attention_mask'])
        
    inputs['input_word_ids']= tf.ragged.constant( inputs['input_word_ids']).to_tensor()
    inputs['input_mask']= tf.ragged.constant( inputs['input_mask']).to_tensor()
    return inputs

In [37]:
df_train_panda_input=input_convert(df_train_panda['masked'].values)
for key in df_train_panda_input.keys():
    df_train_panda_input[key] = df_train_panda_input[key][:,:max_len]

In [38]:
df_dev_panda_input=input_convert(df_dev_panda['masked'].values)
for key in df_dev_panda_input.keys():
    df_dev_panda_input[key] = df_dev_panda_input[key][:,:max_len]

In [39]:
model = build_model()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init_

In [40]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 31)]         0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 31)]         0           []                               
                                                                                                  
 tf_roberta_model_2 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_word_ids[0][0]',         
 odel)                          thPoolingAndCrossAt               'input_mask[0][0]']             
                                tentions(last_hidde                                               
                                n_state=(None, 31,                                          

In [42]:
model.fit(df_train_panda_input,train_data['labels'],verbose=2,validation_data=(df_dev_panda_input,dev_data['labels']),epochs=10) 

Epoch 1/10
187/187 - 46s - loss: 0.1148 - accuracy: 0.9534 - val_loss: 0.3832 - val_accuracy: 0.8872 - 46s/epoch - 243ms/step
Epoch 2/10
187/187 - 45s - loss: 0.0873 - accuracy: 0.9681 - val_loss: 0.4577 - val_accuracy: 0.8844 - 45s/epoch - 238ms/step
Epoch 3/10
187/187 - 45s - loss: 0.0770 - accuracy: 0.9709 - val_loss: 0.5314 - val_accuracy: 0.8758 - 45s/epoch - 239ms/step
Epoch 4/10
187/187 - 45s - loss: 0.0668 - accuracy: 0.9759 - val_loss: 0.4936 - val_accuracy: 0.8730 - 45s/epoch - 239ms/step
Epoch 5/10
187/187 - 44s - loss: 0.0500 - accuracy: 0.9826 - val_loss: 0.5189 - val_accuracy: 0.8806 - 44s/epoch - 238ms/step
Epoch 6/10
187/187 - 44s - loss: 0.0372 - accuracy: 0.9865 - val_loss: 0.5716 - val_accuracy: 0.8834 - 44s/epoch - 238ms/step
Epoch 7/10
187/187 - 44s - loss: 0.0346 - accuracy: 0.9886 - val_loss: 0.5642 - val_accuracy: 0.8796 - 44s/epoch - 238ms/step
Epoch 8/10
187/187 - 44s - loss: 0.0303 - accuracy: 0.9895 - val_loss: 0.6032 - val_accuracy: 0.8844 - 44s/epoch - 238